# 김태영 케라스

In [1]:
import plaidml.keras
plaidml.keras.install_backend()
from tensorflow.keras import backend
from tensorflow.python.keras import backend
import numpy as np
import tensorflow as tf
# from tensorflow import keras
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import array_to_img, img_to_array, load_img

In [2]:
seed = 2020
np.random.seed(seed)
tf.random.set_seed(seed)

## 데이터 부풀리기

In [3]:
# 데이터 부풀리기를 적용한 훈련 데이터 제너레이터
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=15,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.5,
                                   zoom_range=[0.8, 2.0],
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

In [4]:
img = load_img('../dataset/hard_handwriting_shape/train/triangle/triangle001.png')
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

i = 0

# 무한 for loop이니 주의!!!
for batch in train_datagen.flow(x, batch_size=1, save_to_dir='preview',
                                save_prefix='tri', save_format='png'):
    i += 1
    if i > 30: 
        break

## 데이터 셋 생성하기

In [5]:
train_generator = train_datagen.flow_from_directory(
        '../dataset/hard_handwriting_shape/train',
        target_size=(24, 24),
        batch_size=3,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        '../dataset/hard_handwriting_shape/test',
        target_size=(24, 24),    
        batch_size=3,
        class_mode='categorical')

Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


## 모델 구성하기

In [6]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu', input_shape=(24,24,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

INFO:plaidml:Opening device "opencl_amd_ellesmere.0"


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 22, 22, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               819328    
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 387       
Total params: 839,107
Trainable params: 839,107
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

## 모델 학습시키기
- 첫번째 인자 : 훈련데이터셋을 제공할 제네레이터를 지정. 본 예제에서는 앞서 생성한 train_generator으로 지정.
- steps_per_epoch : 한 epoch에 사용한 스텝 수를 지정. 총 45개의 훈련 샘플이 있고 배치사이즈가 3이므로 15 스텝으로 지정.
- epochs : 전체 훈련 데이터셋에 대해 학습 반복 횟수를 지정. 200번을 반복적으로 학습.
- validation_data : 검증데이터셋을 제공할 제네레이터를 지정. 본 예제에서는 앞서 생성한 validation_generator으로 지정.
- validation_steps : 한 epoch 종료 시 마다 검증할 때 사용되는 검증 스텝 수를 지정. 홍 15개의 검증 샘플이 있고 배치사이즈가 3이므로 5 스텝으로 지정.

In [8]:
modelpath = "model/poly.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', 
                               verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=15*50,
        epochs=200,
        callbacks=[checkpointer],
        validation_data=test_generator,
        validation_steps=5)

Epoch 1/200
750/750 [==============================] - 10s 14ms/step - loss: 0.2625 - acc: 0.8996 - val_loss: 1.0638 - val_acc: 0.7333

Epoch 00001: val_loss improved from inf to 1.06385, saving model to model/poly.hdf5
Epoch 2/200
750/750 [==============================] - 7s 9ms/step - loss: 0.0673 - acc: 0.9773 - val_loss: 1.4557 - val_acc: 0.8000

Epoch 00002: val_loss did not improve from 1.06385
Epoch 3/200
750/750 [==============================] - 7s 9ms/step - loss: 0.0342 - acc: 0.9884 - val_loss: 2.0395 - val_acc: 0.8000

Epoch 00003: val_loss did not improve from 1.06385
Epoch 4/200
750/750 [==============================] - 7s 9ms/step - loss: 0.0394 - acc: 0.9862 - val_loss: 1.8417 - val_acc: 0.8000

Epoch 00004: val_loss did not improve from 1.06385
Epoch 5/200
750/750 [==============================] - 7s 9ms/step - loss: 0.0438 - acc: 0.9853 - val_loss: 2.9087 - val_acc: 0.8000

Epoch 00005: val_loss did not improve from 1.06385
Epoch 6/200
750/750 [===================

750/750 [==============================] - 7s 9ms/step - loss: 3.0263e-06 - acc: 1.0000 - val_loss: 8.9022 - val_acc: 0.8000

Epoch 00045: val_loss did not improve from 1.06385
Epoch 46/200
750/750 [==============================] - 7s 9ms/step - loss: 0.0897 - acc: 0.9924 - val_loss: 7.3932 - val_acc: 0.8000

Epoch 00046: val_loss did not improve from 1.06385
Epoch 47/200
750/750 [==============================] - 7s 9ms/step - loss: 0.0093 - acc: 0.9969 - val_loss: 11.5125 - val_acc: 0.7333

Epoch 00047: val_loss did not improve from 1.06385
Epoch 48/200
750/750 [==============================] - 7s 9ms/step - loss: 0.0097 - acc: 0.9982 - val_loss: 11.0685 - val_acc: 0.6667

Epoch 00048: val_loss did not improve from 1.06385
Epoch 49/200
750/750 [==============================] - 7s 9ms/step - loss: 0.0169 - acc: 0.9973 - val_loss: 7.8996 - val_acc: 0.8000

Epoch 00049: val_loss did not improve from 1.06385
Epoch 50/200
750/750 [==============================] - 7s 9ms/step - loss: 0

In [ ]:
del model

In [ ]:
model = load_model('./model/poly.hdf5')

In [ ]:
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

In [ ]:
print("-- Predict --")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

In [ ]:
for i in range(len(test_generator.labels)):
    print(test_generator.filenames[i], test_generator.labels[i], np.argmax(output[i]))